# 📓 Update Strategy Module


In [ ]:
updates = spark.read.format("parquet").load("/lakehouse/updates/parquet_10k")

# Apply CDC logic
spark.sql("MERGE INTO target_table USING updates ON condition WHEN MATCHED THEN UPDATE SET ... WHEN NOT MATCHED THEN INSERT ...")